In [19]:
import torch
from torch import nn
import dltools

In [20]:
def transpose_qkv(X,num_heads):
    X = X.reshape(X.shape[0],X.shape[1],num_heads,-1)
    X = X.permute(0,2,1,3)
    return X.reshape(-1,X.shape[2],X.shape[3])


In [29]:
def transpose_output(X,num_heads):
    X = X.reshape(-1,num_heads,X.shape[1],X.shape[2])
    X = X.permute(0,2,1,3)
    return X.reshape(X.shape[0], X.shape[1], -1)


In [22]:
class MutiHeadAttention(nn.Module):
    def __init__(self, key_size,query_size,value_size,num_hiddens,num_heads,dropout,bia=False, **kwargs):
        super().__init__( **kwargs)
        self.num_heads = num_heads
        self.attention = dltools.DotProductAttention(dropout)
        self.W_q = nn.Linear(query_size,num_hiddens,bias=bia)
        self.W_k = nn.Linear(key_size,num_hiddens,bias=bia)
        self.W_v = nn.Linear(value_size,num_hiddens,bias=bia)
        self.W_o = nn.Linear(num_hiddens,num_hiddens,bias=bia)

    def forward(self,queries,keys,values,valid_lens):
        queries = transpose_qkv(self.W_q(queries),self.num_heads)
        keys = transpose_qkv(self.W_k(keys),self.num_heads)
        values = transpose_qkv(self.W_v(values),self.num_heads)
        if valid_lens is not None:
            valid_lens = torch.repeat_interleave(valid_lens,repeats=self.num_heads,dim=0)

        output = self.attention(queries,keys,values,valid_lens)
        output_concat = transpose_output(output,self.num_heads)

        return self.W_o(output_concat)

In [23]:
num_hiddens = 100
num_heads = 5
attention = MutiHeadAttention(num_hiddens,num_hiddens,num_hiddens,num_hiddens,num_heads,0.2)
attention.eval()

MutiHeadAttention(
  (attention): DotProductAttention(
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (W_q): Linear(in_features=100, out_features=100, bias=False)
  (W_k): Linear(in_features=100, out_features=100, bias=False)
  (W_v): Linear(in_features=100, out_features=100, bias=False)
  (W_o): Linear(in_features=100, out_features=100, bias=False)
)

In [30]:
batch_size,num_queries = 2,4
num_kvparis,valid_lens=6,torch.tensor([3,2])
X= torch.ones((batch_size,num_queries,num_hiddens))
Y = torch.ones((batch_size,num_kvparis,num_hiddens))
attention(X,Y,Y,valid_lens).shape


torch.Size([2, 4, 100])